In [298]:
import pandas as pd
import matplotlib as plt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, text, Column, Integer, String, ForeignKey
from sqlalchemy.orm import Session
from sqlalchemy.orm import declarative_base, relationship
import warnings
warnings.filterwarnings("ignore")


In [306]:
vg_sales = pd.read_csv('./vgsales.csv').set_index('Rank')
vg_sales.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [307]:
vg_sales[vg_sales["Year"].isna()]
len(vg_sales[vg_sales["Year"].isna()])
len(vg_sales[vg_sales["Publisher"].isna()])
vg_sales[vg_sales.isna().any(axis=1)]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
180,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
432,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
471,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
608,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
...,...,...,...,...,...,...,...,...,...,...
16430,Virtua Quest,GC,NaN,Role-Playing,Unknown,0.01,0.00,0.00,0.00,0.01
16496,The Smurfs,3DS,NaN,Action,Unknown,0.00,0.01,0.00,0.00,0.01
16497,Legends of Oz: Dorothy's Return,3DS,2014.0,Puzzle,NaN,0.00,0.01,0.00,0.00,0.01


In [308]:
#removes games with unknown publisher
vg_sales = vg_sales[vg_sales["Publisher"] != "Unknown"]
vg_sales.dropna(how="any", inplace=True)

In [309]:
vg_sales["Year"] = vg_sales["Year"].copy().astype(int)
#vg_sales['Year'] = pd.to_datetime(vg_sales["Year"].astype(str), format="%Y")
vg_sales.head(5)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [310]:
vg_sales[vg_sales["Name"].duplicated() == True]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
24,Grand Theft Auto V,X360,2013,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38
36,Call of Duty: Black Ops II,X360,2012,Shooter,Activision,8.25,4.30,0.07,1.12,13.73
38,Call of Duty: Modern Warfare 3,PS3,2011,Shooter,Activision,5.54,5.82,0.49,1.62,13.46
41,Call of Duty: Black Ops,PS3,2010,Shooter,Activision,5.98,4.44,0.48,1.83,12.73
45,Grand Theft Auto V,PS4,2014,Action,Take-Two Interactive,3.80,5.81,0.36,2.02,11.98
...,...,...,...,...,...,...,...,...,...,...
16589,Secret Files 2: Puritas Cordis,DS,2009,Adventure,Deep Silver,0.00,0.01,0.00,0.00,0.01
16594,Myst IV: Revelation,PC,2004,Adventure,Ubisoft,0.01,0.00,0.00,0.00,0.01
16597,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01


In [313]:
vg_sales[vg_sales['Name'] == 'Grand Theft Auto V']

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
17,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
24,Grand Theft Auto V,X360,2013,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38
45,Grand Theft Auto V,PS4,2014,Action,Take-Two Interactive,3.80,5.81,0.36,2.02,11.98
198,Grand Theft Auto V,XOne,2014,Action,Take-Two Interactive,2.66,2.01,0.00,0.41,5.08
1898,Grand Theft Auto V,PC,2015,Action,Take-Two Interactive,0.36,0.64,0.00,0.08,1.08


In [314]:
plats = pd.DataFrame(vg_sales["Platform"].unique(), columns=["Platform"])
plats.index = plats.index + 1
plats.index.name = "Platform ID"

plats.head()

,Platform
Platform ID,
1,Wii
2,NES
3,GB
4,DS
5,X360


In [320]:
games = pd.DataFrame(vg_sales["Name"].unique(), columns=["Name"])
#games = pd.DataFrame(vg_sales[vg_sales["Name"].unique(),"Genre", "Publisher"], columns=["Name","Genre","Publisher"])
games["Genre"] = vg_sales["Genre"]
games['Publisher'] = vg_sales['Publisher']
games.index = games.index + 1
games.index.name = "Game_ID"
##games["Game_ID"] = games.index
games.head()

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [226]:
#vg_sales.join(games, on="Name")
vg_sales_merged = vg_sales.merge(games, how= 'left', on= 'Name')
vg_sales_merged.index= vg_sales_merged.index + 1
vg_sales_merged.index.name = "Index"
vg_sales_merged.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Game_ID
Index,,,,,,,,,,,
1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,1
2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,2
3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,3
4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,4
5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,5


In [294]:
vg_sales_merged[vg_sales_merged["Name"] == 'Grand Theft Auto V']

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Game_ID
Index,,,,,,,,,,,
17,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40,17
24,Grand Theft Auto V,X360,2013,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38,17
45,Grand Theft Auto V,PS4,2014,Action,Take-Two Interactive,3.80,5.81,0.36,2.02,11.98,17
197,Grand Theft Auto V,XOne,2014,Action,Take-Two Interactive,2.66,2.01,0.00,0.41,5.08,17
1875,Grand Theft Auto V,PC,2015,Action,Take-Two Interactive,0.36,0.64,0.00,0.08,1.08,17


In [228]:
game_plat = pd.DataFrame()

In [289]:
vg_plat_sales = vg_sales_merged.merge(plats, how='left', on="Platform")
game_plat[["Game_ID", "Platform_ID"]] = vg_plat_sales[['Game_ID', 'Platform_ID']]
game_plat.index= game_plat.index + 1
game_plat.index.name = "Index"
# game_plat['Year'] = vg_sales_merged['Year']
game_plat.head()

,Game_ID,Platform_ID,Sales_ID,Year
Index,,,,
23,23.0,2.0,1,2006.0
24,17.0,5.0,2,2006.0
25,24.0,7.0,3,2009.0
26,25.0,9.0,4,1984.0
27,26.0,4.0,5,2005.0


In [231]:
game_plat[game_plat['Game_ID'] == 17 ]

,Game_ID,Platform_ID
Index,,
17,17,6
24,17,5
45,17,11
197,17,18
1875,17,15


In [297]:
sales = vg_plat_sales[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]]
sales.index.name = "Sales_ID"
sales.index = sales.index + 1

sales.head()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Sales_ID,,,,,
1,41.49,29.02,3.77,8.46,82.74
2,29.08,3.58,6.81,0.77,40.24
3,15.85,12.88,3.79,3.31,35.82
4,15.75,11.01,3.28,2.96,33.00
5,11.27,8.89,10.22,1.00,31.37


In [234]:
game_plat[['Sales_ID']] = sales[['Sales_ID']]
game_plat.head()

,Game_ID,Platform_ID,Sales_ID
Index,,,
1,1,1,1
2,2,2,2
3,3,1,3
4,4,1,4
5,5,3,5


In [247]:
engine = create_engine("sqlite:///game_sales.db")

In [248]:
games.to_sql(name="Game", con=engine, if_exists="replace")
plats.to_sql(name="Platform", con=engine, if_exists="replace")
game_plat.to_sql(name="Game_Platform", con=engine, if_exists="replace")
sales.to_sql(name="Sales", con=engine, if_exists="replace")

16191

In [300]:
command = """SELECT Game.Name, Platform.Platform from Game 
join Game_Platform on Game.Game_ID = Game_Platform.Game_ID
join Platform on Platform.Platform_ID = Game_Platform.Platform_ID 
where Game.Game_ID = 17"""

with engine.connect() as conn:
    result = conn.execute(text(command))

    print (result.fetchall())

[('Grand Theft Auto V', 'X360'), ('Grand Theft Auto V', 'PS3'), ('Grand Theft Auto V', 'PS4'), ('Grand Theft Auto V', 'PC'), ('Grand Theft Auto V', 'XOne')]


In [270]:
command = "SELECT * from Game WHERE Game_ID = 17"
with engine.connect() as conn:
    result = conn.execute(text(command))

    print (result.fetchall())

[(17, 'Grand Theft Auto V', 17)]
